In [20]:
from collections import namedtuple, defaultdict
import codecs
from config import config
from bs4 import BeautifulSoup
import pdb
import torch
from nltk.tokenize import word_tokenize
#import tokenizer
import numpy as np
import re
import pickle
import random
##Added by Richard Sun
import en_core_web_sm
nlp = en_core_web_sm.load()
from data_reader import *

In [39]:
TRAIN_DATA_PATH = "data/2014/Restaurants_Train_v2.xml"
TEST_DATA_PATH = "data/2014/Restaurants_Test_Gold.xml"
TRAIN_DATA_PATH = "data/Indonesian/indo_tweets.csv"

# TRAIN_DATA_PATH = "./data/2014/Laptop_Train_v2.xml"
# TEST_DATA_PATH = "./data/2014/Laptops_Test_Gold.xml"

GLOVE_FILE = "../data/word_embeddings/indo_vectors.txt"
OUT_FILE = config.embed_path
DATA_FILE = config.data_path
DIC_FILE = config.dic_path

In [40]:
#!pip install xlrd
import pandas as pd
#help(pd.read_excel)
data = pd.read_csv(TRAIN_DATA_PATH)

In [56]:
data.Sentiment.describe()

count        7132
unique          3
top       Neutral
freq         3091
Name: Sentiment, dtype: object

In [59]:
data[pd.isnull(data[['Sentiment']]).any(axis=1)]

,Unnamed: 0,Username,Keyword,Tweet,Sentiment,Overal Sentiment,Unnamed: 5
6130,6130,ZUL_Hasan,ISIS,TIPS jadi saksi nikah : Jangan baper .. Sesama...,NaN,NaN,remarks: the ISIS in this context does not ref...
6534,6534,tifsembiring,ISIS,"ISIS ...? ada yg bilang ""Istri Sholihah Idaman...",NaN,NaN,remarks: ISIS here does not refer to the Islam...
6538,6538,tifsembiring,HTI,"@rayestu kmbalikan pd hti nurani mas, kita kom...",NaN,NaN,"remarks: hti nurani = hati nurani, does not re..."


In [32]:
data.to_csv('data/Indonesian/indo_tweets.csv')

In [4]:
dr = data_reader(config)
dr.load_data(config.train_path)
dr_valid = data_reader(config, False)
dr_valid.load_data(config.valid_path)
dr_test = data_reader(config, False)
dr_test.load_data(config.test_path)

In [9]:
samples = dr.generate_sample(dr.data_batch)

In [15]:
words = ['ss', 'ff','df','gf']
word_freq_pair = Counter(words)

In [6]:
data_batch = dh.to_batches(data)

defaultdict(<class 'int'>, {2: 805, 0: 2164, 1: 633})


In [8]:
data2 = dh.read(TEST_DATA_PATH, False)

Dataset number: 606
Target error meal
Local Embeddings Saved!


In [60]:
sample = reader.generate_sample(train_batch)
sent_vecs, mask_vecs, label_list, sent_lens = elmo_transform(sample)

In [62]:
sent_vecs, mask_vecs, label_list, sent_lens = elmo_transform([test_batch[0]])

In [97]:
# write to pkl
with open(DATA_FILE, "wb") as f:
    pickle.dump([train_batch, test_batch],f)

In [98]:
with codecs.open(DIC_FILE, 'wb') as f:
    pickle.dump(reader.id2word, f)

In [99]:
reader.gen_vectors_glove()

1411 unk out of 5138 vocab


In [54]:
from model_att import *
model = attTSA(config)

/Users/richardsun/anaconda3/envs/allennlp/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [107]:
doc = nlp(u"To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora.")
target = 'food'
for token in doc:
#     if 'food' in token.head.text:
#         print(token)
#     children = [t.text for t in token.children]    
#     if 'food' in children:
#         print(token)
    print('Text:'+ token.text, 'Head:'+token.head.text, 'Children:',
           [child for child in token.children])
#     if token.text == 'food':
#         a = token
#     if token.text == 'deficiencies':
#         b= token
    

Text:To Head:be Children: []
Text:be Head:was Children: [To, fair]
Text:completely Head:fair Children: []
Text:fair Head:be Children: [completely]
Text:, Head:was Children: []
Text:the Head:factor Children: []
Text:only Head:factor Children: []
Text:redeeming Head:factor Children: []
Text:factor Head:was Children: [the, only, redeeming]
Text:was Head:was Children: [be, ,, factor, food, ,, but, make, .]
Text:the Head:food Children: []
Text:food Head:was Children: [the, ,, was]
Text:, Head:food Children: []
Text:which Head:was Children: []
Text:was Head:food Children: [which, above]
Text:above Head:was Children: [average]
Text:average Head:above Children: []
Text:, Head:was Children: []
Text:but Head:was Children: []
Text:could Head:make Children: []
Text:n't Head:make Children: []
Text:make Head:was Children: [could, n't, up, for]
Text:up Head:make Children: []
Text:for Head:make Children: [deficiencies]
Text:all Head:deficiencies Children: []
Text:the Head:deficiencies Children: []
Tex

In [103]:
for item in doc.sents:
    print(item)
    print('*******')

To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora.
*******


In [46]:
doc = nlp(u'the screen is good but the battery sucks')
for token in doc:
#     if 'food' in token.head.text:
#         print(token)
#     children = [t.text for t in token.children]    
#     if 'food' in children:
#         print(token)
    print('Text:'+ token.text, 'Head:'+token.head.text, 'Children:',
           [child for child in token.children])

Text:the Head:screen Children: []
Text:screen Head:is Children: [the]
Text:is Head:is Children: [screen, good, but, sucks]
Text:good Head:is Children: []
Text:but Head:is Children: []
Text:the Head:battery Children: []
Text:battery Head:sucks Children: [the]
Text:sucks Head:is Children: [battery]


In [44]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)
# trees = doc.print_tree()
# trees[0]

In [1]:
from Layer import SimpleCat
from data_reader_general import dataHelper, data_generator
#from model_glove import *
from model_crf import *
from config import config
cat_layer = SimpleCat(config)

In [2]:
#cat_layer.load_vector()

In [189]:
sent = 'To heal the hearts of believers and to enrage kuffar, \
munafikeen and murtadeen, we present the last video memoir of Shaheed Eisa Fazili. May Allah accept him into highest Jannah'
targets = ['Eisa Fazili', 'kuffar', 'munafikeen', 'murtadeen', 'Allah']

In [164]:
sent = "But the staff was so good to us"
targets = ['staff']

In [190]:
sentence_list = []
opinion_list = []
SentInst = namedtuple("SentenceInstance", "id text text_ids text_inds opinions")
OpinionInst = namedtuple("OpinionInstance", "target_text polarity class_ind target_mask target_ids")
for target in targets:
    opinion_inst = OpinionInst(target, 'negative', None, None, None)
    opinion_list.append(opinion_inst)
sent_Inst = SentInst(1, sent, None, None, opinion_list)
sentence_list.append(sent_Inst)

In [191]:
dh = dataHelper(config)
data, words = dh.process_raw_data(sentence_list)

In [192]:
data_batch = dh.to_batches(data)

defaultdict(<class 'int'>, {2: 5})


In [193]:
dg = data_generator(config, data_batch)

In [194]:
sent_vecs, mask_vecs, label_list, sent_lens = dg.elmo_transform(data_batch)

In [195]:
#model = AspectSent(config)

In [197]:
pred_label, best_seq = model.predict(sent_vecs, mask_vecs, sent_lens) 

In [198]:
pred_label

tensor([ 0,  0,  0,  0,  0])

In [199]:
scores, s_probs, best_seqs = model.compute_predict_scores(sent_vecs, mask_vecs, sent_lens)

In [200]:
scores

tensor([[ 2.6032, -4.7932,  0.9438],
        [ 2.6839, -5.2023,  1.0347],
        [ 2.7342, -5.2895,  1.0273],
        [ 2.7248, -5.2792,  1.0532],
        [ 2.7875, -5.3021,  1.0744]])

In [201]:
import torch.nn.functional as F
F.softmax(scores, dim=1)

tensor([[ 0.8397,  0.0005,  0.1598],
        [ 0.8385,  0.0003,  0.1612],
        [ 0.8462,  0.0003,  0.1535],
        [ 0.8415,  0.0003,  0.1582],
        [ 0.8470,  0.0003,  0.1527]])

In [203]:
from data_reader import data_reader
dr = data_reader(config)
dr.load_data(config.train_path)

In [205]:
dr.data_len

3001

In [210]:
token_list, mask_list, label_list, _ = zip(*dr.data_batch)

In [212]:
unique, counts = np.unique(label_list, return_counts=True)

In [213]:
counts

array([1816,  531,  654])

In [214]:
unique

array([0, 1, 2])

In [1]:
import torch

In [2]:
a = [[0,1,1,0],[0,1,0], [0,0,0,1,1]]


In [3]:
b = torch.zeros(3, 5)

In [5]:
for i in range(3):
    temp = a[i]
    b[i, :len(temp)] = torch.FloatTensor(temp)

In [36]:
a=torch.LongTensor([2,5,7,3,4])

In [37]:
sent_lens, perm_idx = a.sort(0, descending=True)

In [41]:
a[perm_idx[0].item()]

tensor(7)

In [38]:
sent_lens

tensor([ 7,  5,  4,  3,  2])

In [1]:
from __future__ import division
from model_att_glove import *
from data_reader_general import *
from config import config
import pickle
from Layer import GloveMaskCat
import numpy as np
import codecs
import copy
import os, sys

In [2]:
save_path = 'data/22222222222222222222222.txt'
data = np.random.randn(10,20)
with open(save_path, "wb") as f:
    pickle.dump(data,f)
print('Saving successfully!')

Saving successfully!


In [2]:
dr = data_reader(config)
train_data = dr.load_data(config.data_path+'Restaurants_Train_v2.xml.pkl')
test_data = dr.load_data(config.data_path+'Restaurants_Test_Gold.xml.pkl')
valid_data = dr.load_data(config.data_path+'valid_restaurant.pkl')
dg_train = data_generator(config, train_data)
dg_valid = data_generator(config, valid_data, False)
dg_test =data_generator(config, test_data, False)

In [3]:
cat_layer = GloveMaskCat(config)
cat_layer.load_vector()

Loaded from data/2014/vocab/local_emb.pkl with shape (4334, 300)


In [4]:
sent_vecs, mask_vecs, label_list, sent_lens, tokens = next(dg_valid.get_ids_samples())

In [5]:
mask_vecs[0]

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0])

In [6]:
sent_vecs[0]

tensor([   52,  2175,    11,  2176,   220,   102,    10,     3,   134,
          983,  2177,     3,   451,  2178,     1,  2179,    11,   152,
          834,   233,   218,    44,   260,    32,  1482,    37,     4,
          511,    32,  2180,  2181,    38,     6,   116,  1184,    13,
            1,   141,   107,     4,   984,     2])

In [10]:
def visualize_attention(dr_test, model):
    print("Evaluting")
    dr_test.reset_samples()
    model.eval()
    all_counter = 0
    correct_count = 0
    while dr_test.index < dr_test.data_len:
        sent, mask, label, sent_len, tokens = next(dr_test.get_ids_samples())
        sent, target = cat_layer(sent, mask)
        if config.if_gpu: 
            sent, target = sent.cuda(), target.cuda()
            label, sent_len = label.cuda(), sent_len.cuda()
        pred_label, attentions  = model.predict(sent, target, sent_len) 
        print(tokens[:3])
        print(mask[:3])
        print(attentions[:3])
        break

In [11]:
model_file = config.model_path+'model.pt'
if os.path.exists(model_file):
    model = torch.load(model_file)
    visualize_attention(dg_valid, model)

Evaluting
[['an', 'oasis', 'of', 'refinement', ':', ' ', 'food', ',', 'though', 'somewhat', 'uneven', ',', 'often', 'reaches', 'the', 'pinnacles', 'of', 'new', 'american', 'fine', 'cuisine', '  ', 'chef', "'s", 'passion', '(', 'and', 'kitchen', "'s", 'precise', 'execution', ')', 'is', 'most', 'evident', 'in', 'the', 'fish', 'dishes', 'and', 'soups', '.'], ['the', 'food', 'can', 'get', 'pricey', 'but', 'the', 'prixe', 'fixe', 'tasting', 'menu', 'is', 'the', 'greatest', 'food', 'for', 'a', 'good', 'price', 'and', 'they', 'cater', 'the', 'food', 'to', 'any', 'food', 'allergies', 'or', 'food', 'you', 'do', "n't", 'like', '.'], ['the', 'red', 'curry', 'is', 'weak', 'and', 'tasteless', ',', 'the', 'pad', 'thai', 'is', 'stuck', 'together', 'and', 'lumpy', ',', 'the', 'rice', 'is', 'often', 'overcooked', ',', 'and', 'the', 'seafood', 'is', 'pretty', 'sketchy', '.']]
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [37]:
import numpy as np
emb_dim= 300
def load_pretrained_word_emb(file_path):
    '''
    Load a specified vocabulary
    '''
    word_emb = {}
    vocab_words = set()
    with open(file_path) as fi:
        for line in fi:
            items = line.split()
            word = ' '.join(items[:-1*emb_dim])
            vec = items[-1*emb_dim:]
            word_emb[word] = np.array(vec, dtype=np.float32)
            vocab_words.add(word)
    return word_emb, vocab_words

In [38]:
file = '../data/glove.840B.300d.txt'
a,b = load_pretrained_word_emb(file)

KeyboardInterrupt: 

In [33]:
a ='hello -0.1573 -0.29517 0.30453 -0.54773 0.098293 -0.1776 0.21662 0.19261 -0.21101 0.53788 -0.047755 0.40675 0.023592 -0.32814 0.046858 0.19367 0.25565 -0.021019 -0.15957 -0.1023 0.20303 -0.043333 0.11618 -0.18486 0.0011948 -0.052301 0.34587 0.052335 0.16774 -0.21384 0.055947 0.24934 -0.12179 0.16749 0.28922 -0.033739 0.3015 -0.13241 0.092635 0.37155 -0.2884 -0.0052731 -0.001005 -0.51153 -0.28476 -0.20139 0.11837 -0.0055891 0.43604 0.16796 -0.2701 0.063957 -0.093253 -0.22079 0.36501 0.06545 0.23941 -0.19292 0.098293 0.12172 -0.1168 -0.027436 0.20507 -0.39139 -0.23111 0.46239 0.22888 -0.028415 -0.1798 0.23817 0.28093 -0.47935 0.23177 -0.35587 0.14246 0.11861 0.011018 0.091986 0.0054809 -0.39955 -0.40183 -0.10629 -0.30851 0.12383 -0.16737 -0.43569 0.4211 -0.57416 -0.19964 0.51312 0.090747 -0.21657 0.043519 0.24288 0.081134 0.49104 -0.33342 -0.31056 -0.3136 0.26931 -0.14402 0.33185 -0.21662 -0.072985 0.080603 -0.7266 -0.098385 -0.36233 -0.25346 0.1154 0.25738 0.15802 -0.15633 -0.024581 0.35673 0.31153 0.33475 -0.081155 -0.3061 0.019077 -0.049047 -0.11232 -0.07417 0.35596 -0.2642 0.012781 -0.20715 0.020223 0.054534 -0.28803 0.42863 -0.10312 0.24771 0.013196 0.19768 -0.013528 -0.15134 0.20307 -0.028973 -0.022706 -0.29199 -0.082062 0.19048 0.0053574 0.14067 -0.28675 0.21343 0.42428 -0.28186 -0.11801 -0.45227 -0.0067998 0.044784 -0.0062886 0.25087 0.34481 -0.64459 -0.20467 0.35007 0.1468 -0.14007 -0.0050219 -0.24053 0.41426 -0.40902 0.21141 0.25726 -0.4883 0.027066 0.56367 -0.39594 -0.035206 0.63079 0.14343 0.038315 0.32527 -0.080335 -0.20065 -0.30848 -0.0031591 0.15296 -0.21014 0.42143 -0.20944 -0.069285 0.13555 -0.020401 -0.22555 0.33491 0.16035 0.17739 -0.023627 0.097575 -0.19395 -0.018754 -0.119 -0.0067027 -0.4178 0.29027 0.13034 -0.30212 0.61173 -0.39918 -0.020191 -0.34531 -0.092082 0.46818 0.36671 0.21021 -0.053162 -0.37872 -0.14271 -0.13604 0.31715 -0.17227 -0.091266 0.16417 0.15069 0.53556 -0.29678 0.13965 -0.29788 0.1282 0.1971 -0.045515 -0.41355 -0.050333 -0.39015 -0.29579 -0.096145 -0.03151 0.053714 -0.37309 -0.36523 -0.17235 0.39251 -0.065909 -0.25267 -0.34448 -0.11503 0.43665 0.18832 0.20631 0.27801 -0.046077 0.13397 -0.091953 -0.098542 0.15811 0.2752 0.081383 0.32077 -0.10028 0.1088 -0.24836 0.10477 0.15243 -0.071302 0.12861 0.23061 0.0074864 0.090918 -0.12269 -0.14831 0.010586 0.35745 -0.23412 -0.23746 -0.22646 -0.27641 -0.1634 0.071909 -0.093884 0.21331 -0.20627 0.44406 0.34691 0.019064 0.034657 0.36789 0.32276 -0.31099 -0.023443 -0.77048 -0.26001 0.033961 -0.13874 0.051973 -0.0090509 0.27427 0.046548 -0.48214 -0.1437 -0.1975 -0.038126 -0.16555 0.071697 0.049449 0.15386 -0.81663'

In [74]:
from model_parse_glove import *
from data_reader_general import *
from parse_path import dependency_path
from configs.config_parse import config
import pickle
from Layer import GloveMaskCat
import numpy as np
import codecs
import copy
import os, sys

ModuleNotFoundError: No module named 'allennlp'

In [7]:
from parse_path import dependency_path
dp = dependency_path()
import torch
def get_dependency_weight(tokens, targets):
    max_len = max(map(len, tokens))
    weights = np.zeros([len(tokens), max_len])
    for i, token in enumerate(tokens):
        #print('Original word num')
        #print(len(token))
        print(i)
        text = ' '.join(token)#Connect them into a string

        graph = dp.build_graph(text)
        mat = dp.compute_node_distance(graph)
        #print(len(new_tokens))
        if len(token) != len(mat):
            print('Word number conflicts!')

        try:
            max_w, _, _ = dp.compute_soft_targets_weights(mat, targets[i])
            weights[i, :len(max_w)] = max_w
        except:
            print('text process error')
            print(text, targets[i])
            break
    return torch.FloatTensor(weights)

In [84]:
tokens = [['i', 'understand', 'the', 'area', 'and', 'folks', 'you', 'need', 'not', 'come', 'here', 'for', 'the', 'romantic', ',', 'alluring', 'ambiance', 'or', 'the', 'five', 'star', 'service', 'featuring', 'a', 'sommlier', 'and', 'a', 'complicated', 'maze', 'of', 'captain', 'and', 'back', 'waiters', 'you', 'come', 'for', 'the', 'authentic', 'foods', ',', 'the', 'tastes', ',', 'the', 'experiance', '.'], ['the', 'dishes', 'offered', 'were', 'unique', ',', 'very', 'tasty', 'and', 'fresh', 'from', 'the', 'lamb', 'sausages', ',', 'sardines', 'with', 'biscuits', ',', 'large', 'whole', 'shrimp', 'to', 'the', 'amazing', 'pistachio', 'ice', 'cream', '(', 'the', 'best', 'and', 'freshest', 'i', "'ve", 'ever', 'had', ')', '.'], ['i', 'am', 'a', '100', 'lb', 'girl', ',', 'had', 'a', 'glass', 'of', 'wine', 'and', 'a', 'glass', 'of', 'beer', 'prior', 'to', 'the', 'dinner', ',', 'and', 'i', 'was', 'still', 'hungry', 'after', 'my', 'visit', 'to', 'this', 'place'], ['the', 'people', 'with', 'carts', 'of', 'food', 'do', "n't", 'understand', 'you', 'because', 'they', 'do', "n't", 'speak', 'english', ',', 'their', 'job', 'is', 'to', 'give', 'you', 'the', 'delicious', 'food', 'you', 'point', 'at', '.'], ['i', 'ordered', 'tamarind', 'duck','and', 'my', 'wife', 'ordered', 'noodles', 'with', 'ground', 'beef', ',', 'and', 'we', 'were', 'both', 'delighted', 'by', 'the', 'way', 'the', 'dishes', 'evoked', 'thai', 'flavors', 'in', 'unexpected', 'ways', '.'], ['if', 'you', "'re", 'looking', 'for', 'perfect', 'traditional', 'sushi', ',', 'go', 'here', 'if', 'you', "'re", 'looking', 'for', 'interesting', 'combinations', ',', 'try', 'sushi', 'of', 'gari', "'s", '(', 'east', 'side', ')', '.'], ['i', 'have', 'lived', 'in', 'japan', 'for', '7', 'years', 'and', 'the', 'taste', 'of', 'the', 'food', 'and', 'the', 'feel', 'of', 'the', 'restaurant', 'is', 'like', 'being', 'back', 'in', 'japan', '.'], ['yes', ',', 'they', 'use', 'fancy', 'ingredients', ',', 'but', 'even', 'fancy', 'ingredients', 'do', "n't", 'make', 'for', 'good', 'pizza', 'unless', 'someone', 'knows', 'how', 'to', 'get', 'the', 'crust', 'right', '.'], ['we', 'recently', 'decided', 'to', 'try', 'this', 'location', ',', 'and', 'to', 'our', 'delight', ',', 'they', 'have', 'outdoor', 'seating', ',', 'perfect', 'since', 'i', 'had', 'my', 'yorkie', 'with', 'me', '.'], ['succulent', 'steaks', 'cooked', 'precisely', 'to', 'your', 'desired', "'", 'doneness', "'", 'accompanied', 'by', 'salads', 'and', 'sides', 'that', 'do', "n't", 'look', 'like', 'leafy', 'road', 'kill', '.'], ['i', 'usually', 'go', 'there', 'later', 'at', 'night', 'when', 'i', 'get', 'off', 'work', 'so', 'i', 'do', "n't", 'have', 'to', 'deal', 'with', 'crowds', 'or', 'lines', '.'], ['the', 'space', 'is', 'a', 'bit', 'too', 'small', 'for', 'live', 'music', ',', 'so', 'on', 'jazz', 'nights', ',', 'it', 'can', 'be', 'loud', 'and', 'cramped', '.'], ['the', 'real', 'kicker', 'of', 'the', 'menu', ',', 'however', ',', 'is', 'the', 'beef', 'cubes', 'or', 'the', 'chicken', 'with', 'chili', 'and', 'lemon', 'grass', '.'], ['waiters', 'tend', 'to', 'forget', 'drinks', 'completely', ',', 'food', 'portions', 'are', 'so', 'tiny', ',', 'two', 'people', 'have', 'trouble', 'sharing', 'one', 'entree', '.'], ['any', 'if', 'you', 'have', 'a', 'reservation', 'you', "'ll", 'wait', 'for', 'max', '5', 'minutes', 'so', 'have', 'a', 'drink', 'at', 'the', 'bar', '.'], ['a', 'restaurant', 'that', 'does', "n't", 'try', 'to', 'do', 'anything', 'except', 'serve', 'great', 'food', 'with', 'great', 'service', 'in', 'a', 'pleasant', 'atmosphere', '.'], ['the', 'in', 'house', 'lady', 'dj', 'on', 'saturday', 'nights', 'has', 'outrageously', 'good', 'taste', 'in', 'music', ',', 'and', 'moreover', ',', 'takes', 'requests', '.'], ['the', 'wine', 'is', 'always', 'good', ',', 'the', 'tapas', 'are', 'always', 'yummy', ',', 'especially', 'with', 'the', 'warm', 'pita', 'bread', '.'], ['only', 'drawback', 'they', 'wo', "n't", 'toast', 'your', 'bagel', ',', 'and', 'they', 'do', "n't", 'make', 'eggs', 'for', 'the', 'bagel', '.'], ['one', 'of', 'my', 'favorites', 'though', 'was', 'the', 'angry', 'lobster', ',', 'a', 'cold', 'lobster', 'salad', 'that', 'was', 'magnificent', '.'], ['the', 'food', 'is', 'wonderful', ',', 'tasty', 'and', 'filling', ',', 'and', 'the', 'service', 'is', 'professional', 'and', 'friendly', '.'], ['overall', 'the', 'restaurant', 'is', 'more', 'expensive', 'than', 'our', 'other', 'sushi', 'favorites', ',', 'but', 'everything', 'was', 'delicious', '.'], ['try', 'the', 'pad', 'se', 'ew', 'or', 'chicken', 'with', 'cashew', 'nuts', 'for', 'a', 'memorable', 'and', 'repeatable', 'experience', '.'], ['food', 'is', 'average', ',', 'and', 'i', 'would', 'say', 'even', 'the', 'chain', 'restaurant', 'baluchi', "'s", 'tastes', 'better', '.'], ['after', 'my', '3rd', 'time', 'the', 'manager', 'remembered', 'me', 'and', 'treated', 'me', 'like', 'an', 'usual', 'customer', '.'], ['they', 'would', 'nt', 'even', 'let', 'me', 'finish', 'my', 'glass', 'of', 'wine', 'before', 'offering', 'another', '.'], ['they', 'even', 'have', 'a', 'section', 'in', 'the', 'menu', 'called', 'american', 'chinese', 'food'], ['the', 'wait', 'staff', 'is', 'very', 'friendly', ',', 'if', 'not', 'overly', 'efficient', '.'], ['i', 'could', "n't", 'reccommend', 'their', 'godmother', 'pizza', 'any', 'higher', '.'], ['yeah', ',', 'sometimes', 'the', 'service', 'can', 'be', 'slow', '.'], ['tasty', 'steak', ',', 'pork', 'loin', ',', 'the', 'works', '.'], ['service', 'friendly', 'and', 'attentive', '.']]

In [85]:
targets = [[1,2]]*20

In [6]:
a = 'Great food, great prices, great service.'

In [10]:
import spacy
spanlp = spacy.load('en')

In [62]:
def convert_mask_index(masks):
    '''
    Find the indice of none zeros values in masks, namely the target indice
    '''
    target_indice = []
    for mask in masks:
        indice = torch.nonzero(mask == 1).squeeze(1).numpy()
        target_indice.append(indice)
    return target_indice

In [25]:
import networkx as nx
text = "same owner as the guy who owns typhoon , which is just down the street on St. marks and 1st ave ."
document = spanlp(text.replace(" '", "'"))
edges = []
for token in document:
    # FYI https://spacy.io/docs/api/token
    for child in token.children:
        edges.append(('{0}-{1}'.format(token.lower_,token.i),
                    '{0}-{1}'.format(child.lower_,child.i)))

graph = nx.Graph(edges)
graph.nodes()

NodeView(('owner-1', 'same-0', 'as-2', ',-8', 'is-10', '.-21', 'guy-4', 'the-3', 'owns-6', 'who-5', 'typhoon-7', 'which-9', 'down-12', 'just-11', 'street-14', 'the-13', 'on-15', 'marks-17', 'st.-16', 'and-18', 'ave-20', '1st-19'))

In [4]:
with open('data/ppdb-2.0-s-all') as f:
    i =0 
    for line in f:
        i += 1
        print(line)
        if i>3:
            break

[NN] ||| transplant ||| transplantation ||| PPDB2.0Score=5.24981 PPDB1.0Score=3.295900 -logp(LHS|e1)=0.18597 -logp(LHS|e2)=0.14031 -logp(e1|LHS)=11.83583 -logp(e1|e2)=1.80507 -logp(e1|e2,LHS)=1.46728 -logp(e2|LHS)=11.47593 -logp(e2|e1)=1.49083 -logp(e2|e1,LHS)=1.10738 AGigaSim=0.63439 Abstract=0 Adjacent=0 CharCountDiff=5 CharLogCR=0.40547 ContainsX=0 Equivalence=0.371472 Exclusion=0.000344 GlueRule=0 GoogleNgramSim=0.03067 Identity=0 Independent=0.078161 Lex(e1|e2)=9.64663 Lex(e2|e1)=59.48919 Lexical=1 LogCount=4.67283 MVLSASim=NA Monotonic=1 OtherRelated=0.372735 PhrasePenalty=1 RarityPenalty=0 ForwardEntailment=0.177287 SourceTerminalsButNoTarget=0 SourceWords=1 TargetComplexity=0.98821 TargetFormality=0.98464 TargetTerminalsButNoSource=0 TargetWords=1 UnalignedSource=0 UnalignedTarget=0 WordCountDiff=0 WordLenDiff=5.00000 WordLogCR=0 ||| 0-0 ||| OtherRelated

[JJ] ||| <www.un.org/depts/dgacm/docs/crp/aconf212crp1/russian.pdf> ||| <www.un.org/depts/dgacm/docs/crp/aconf212crp1/arabic

In [3]:
line.split('|||')

['[NN] ',
 ' transplant ',
 ' transplantation ',
 ' PPDB2.0Score=5.24981 PPDB1.0Score=3.295900 -logp(LHS|e1)=0.18597 -logp(LHS|e2)=0.14031 -logp(e1|LHS)=11.83583 -logp(e1|e2)=1.80507 -logp(e1|e2,LHS)=1.46728 -logp(e2|LHS)=11.47593 -logp(e2|e1)=1.49083 -logp(e2|e1,LHS)=1.10738 AGigaSim=0.63439 Abstract=0 Adjacent=0 CharCountDiff=5 CharLogCR=0.40547 ContainsX=0 Equivalence=0.371472 Exclusion=0.000344 GlueRule=0 GoogleNgramSim=0.03067 Identity=0 Independent=0.078161 Lex(e1|e2)=9.64663 Lex(e2|e1)=59.48919 Lexical=1 LogCount=4.67283 MVLSASim=NA Monotonic=1 OtherRelated=0.372735 PhrasePenalty=1 RarityPenalty=0 ForwardEntailment=0.177287 SourceTerminalsButNoTarget=0 SourceWords=1 TargetComplexity=0.98821 TargetFormality=0.98464 TargetTerminalsButNoSource=0 TargetWords=1 UnalignedSource=0 UnalignedTarget=0 WordCountDiff=0 WordLenDiff=5.00000 WordLogCR=0 ',
 ' 0-0 ',
 ' OtherRelated\n']